## Letter Linking

Info encoded:
    
    word length k in free group on n letters : 
            list of length k, 
            elements are lists of integers of length 2, 
            [i][0] = 1, ... , n
            [i][1] = -1, +1 
            example: [ [1,-1] , [1,-1] , [4,1] , [3,-1] ] is word $ X_1^{-2} X_4 X_3^{-1} $ 
            
    Function on a word:
            list of integers the same length as the word
            
    1-Form on a word:
            list of integers the same length as the word which is anti symmetric
            
    Symbols:
            Strings '(1)((2)3)2' = 1--> 2 <-- 3 <--2
            
Functions:

    dx(word):
        returns the cannonical 1-form on this word
        
    chi(k)(word):
        returns the indicator 1-form on the word with respect to the k^th letter X_k
        
     isForm(word, 1-form):
         returns boolean if 1-form is valid on the word
             (not used currently)
             
     dInv(word, 1-form):
         returns the function which is d^{-1}(form)
         
     concatenate(list of strings):
         returns string which is concatenation of the elements in the list of strings
         
     letterLink(symbol, word):
         returns the 1-form on the word as prescribed by the symbol
             IDEA: recursive function 
                 ~if symbol = '1',...,'n': return appropriate indicator form
                 ~if symbol = '(inside)': return dInv(word , letterLink('inside' ,word))
                 ~else: break symbol into concatenation of symbols of the previous type 
                        find product of letterLink of sub symbols
                         i.e. '(1)((2)1)4' --> [ '(1)' , '((2)1)' , '4' ]
                              return letterLink('(1)', w) * letterLink('((2)1)', w) * letterLink('4', w)
         
      

In [1]:
#Starting info
numGen = 4

In [184]:
import numpy as np

In [185]:
##standard 1-form dx on w
def dx(word):
    return np.array([ word[i][1] for i in range(len(word)) ])


In [186]:
##indicator forms on w
#creates the function chi_k which is the indicator 1-form for k^th letter
#that is chi_k ( in : word ; out : list of signed occurances of x_k )
def chi(k):
    if k not in range(1 , numGen +1):
        return lambda w : "Error: indicator variable out of range"
    else:
        return lambda w : np.array([w[i][1] if w[i][0] == k else 0 for i in range(len(w)) ])

In [149]:
##function which checks if list is 1-form or function with respect to a word
def isForm(word, form):
    #check if they have the same length
    if len(word) != len(form):
        print("\nword and form are not the same length")
        return False
    #For each letter in alphabet find (with sign) occurances in word
    enWord = list(enumerate(word))
    for i in range(1,numGen+1):
        posi = [index for (index , item) in enWord if item == [i,1]] 
        negi = [index for (index , item) in enWord if item == [i,-1]]
        
        #for the i^th letter in alphabet, check that these lists are appropriate
        ######## POTENTIAL SPEED UP HERE: ONLY NEED TO CHECK HALF OF CARTESIAN PRODUCT
        for k in cartesian_product([posi,posi]):
            if form[k[0]] != form[k[1]]:
                print(f'\nvalue of form on letter X_{i} is not constant')
                return False
            
        for k in cartesian_product([negi, negi]):
            if form[k[0]] != form[k[1]]:
                print(f'\nvalue of form on letter X_{i}^-1 is not constant')
                return False
            
        for k in cartesian_product([posi,negi]):
            if form[k[0]] != -form[k[1]]:
                print(f'\nform is not antisymmetric with respect to involution')
                return False
    return True    
            
            

In [252]:
##anti derivative of 1-form
def dInv( word, form ):
    #make sure form is an array
    form = np.array(form)
    
    ####### muting this check for the time being
    
    #check that form and word are compatible (i.e. form is skew symmetric )
    #if isForm(word, form) == False:
     #   raise TypeError('form and word are incompatible')
    
    ######
    
    #find unique function f so that form = f dx
    f = form * dx(word)
    #create list epsilon
    epsilon = [1 if word[i][1] == 1 else 0 for i in range(len(word))]
    #with f calculate the anti derivative
    return np.array([sum(form[:i]) + epsilon[i]*f[i] for i in range(len(word))])

In [293]:
#List of strings to string function
def concatenate(l):
    tempString = ''
    for t in range( len(l)):
        tempString = tempString + l[t] 
    return tempString

In [334]:
##letter link
#in : symbol and word ; out : list for letter link
#example: ( "(1)2" , [x_1, x_2] ) --> [0,1,0,0]
def letterLink(symbol , word):
    w =word
    #clean string and enumerate
    s = symbol.strip()
    e = list(s)
    
    
    #####These are the 'atomic symbols'
    #for symbols of length 1 return the corresponding indicator form
    if len(e) ==1 :
        return chi(int(e[0]))(w)
    
    #for symbols begining and ending with parenthases return dInverse of the letter link of the inside
    if e[0] == '(' and e[len(e)-1] == ')':
        #extract the string in between the parenthases
        tempString = concatenate(e[1:len(e)-1])
        # dInv of the form inbetween the parenthases
        return dInv(w, letterLink(tempString , w))
    
    #####General symbol broken into a product of the above 
    else:
        #find position of the outermost parenthases
        #make a list which keeps track of the parenthases as we iterate through
        pCounter = []
        count = 0
        for i in range(len(e)):
            if e[i] == '(':
                count = count +1
            if e[i] == ')':
                count = count -1
            pCounter.append(count)
  
        #find where count of parenthases are zero
        zeroCount = [ index  for index, entry in enumerate(pCounter) if entry == 0]
       
        #based of zero counts make sub lists of symbol for outermost components
        pLocation = []
        pLocation.append([0,zeroCount[0]])
        for i in range(1,len(zeroCount)):
            pLocation.append( [zeroCount[i-1]+1 , zeroCount[i]])
        
        #product of the letterLink of these substrings
        prod = np.array([1 for i in range(len(w))]) 
        for k in pLocation:
            prod = prod*letterLink(concatenate(e[k[0]:k[1]+1]),w)
        return prod

## Example

we calculate the letter linking of the word $a^2 b a^{-2} c^{-1} a^{-1} b^2 c$ on the symbol $((A)B)C$. Note that exponents which are  $\ne \pm 1$ must be encoded as repreated letters.

In [338]:
w = [[1,1] ,[1,1] ,[2,1] , [1,-1] ,[1,-1], [3,-1], [1,-1],[2,1], [2,1], [1,1]]

s = '((1)2)3'

letterLink(s , w)

array([ 0,  0,  0,  0,  0, -2,  0,  0,  0,  0])